# Modeling

In [1]:
from datetime import datetime
start_time = datetime.now()

# Libraries to load
# !pip install lightgbm

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import lightgbm as lgb
import numpy as np
%matplotlib inline
import time
print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:05.449087


In [2]:
%cd "D:\USC\4th semester\DSO562 fraud"

D:\USC\4th semester\DSO562 fraud


In [3]:
%%time
vars = pd.read_csv('vars.csv')

Wall time: 1min 16s


In [4]:
vars['fraud_label'].head()

0    0
1    1
2    0
3    0
4    0
Name: fraud_label, dtype: int64

In [5]:
# use this to cap variables. For some problems it helps
Clip = 10

In [6]:
vars['fraud_label'].sum()

14393

In [7]:
# top 30 variables from hw7
vars_keep=['address_count_30',
 'fulladdress_count_30',
 'fulladdress_homephone_count_30',
 'ssn_dob_count_30',
 'name_dob_count_30',
 'ssn_count_30',
 'ssn_firstname_count_30',
 'ssn_name_count_30',
 'name_dob_count_14',
 'ssn_count_14',
 'ssn_name_dob_count_14',
 'ssn_lastname_count_14',
 'ssn_name_count_14',
 'ssn_dob_count_0_by_30',
 'name_dob_count_0_by_30',
 'ssn_count_0_by_30',
 'ssn_name_dob_count_0_by_30',
 'homephone_count_3',
 'name_dob_count_0_by_14',
 'ssn_dob_count_0_by_14',
 'ssn_count_7',
 'ssn_name_dob_count_0_by_14',
 'ssn_name_count_7',
 'fulladdress_homephone_count_3',
 'ssn_count_3',
 'ssn_firstname_count_3',
 'ssn_dob_count_3',
 'ssn_name_count_3',
 'ssn_lastname_count_3',
 'ssn_name_dob_count_3',
 'fraud_label']

In [8]:
vars = vars.filter(vars_keep, axis=1)

In [9]:
vars.shape

(1000000, 31)

In [12]:
vars.columns

Index(['address_count_30', 'fulladdress_count_30',
       'fulladdress_homephone_count_30', 'ssn_dob_count_30',
       'name_dob_count_30', 'ssn_count_30', 'ssn_firstname_count_30',
       'ssn_name_count_30', 'name_dob_count_14', 'ssn_count_14',
       'ssn_name_dob_count_14', 'ssn_lastname_count_14', 'ssn_name_count_14',
       'ssn_dob_count_0_by_30', 'name_dob_count_0_by_30', 'ssn_count_0_by_30',
       'ssn_name_dob_count_0_by_30', 'homephone_count_3',
       'name_dob_count_0_by_14', 'ssn_dob_count_0_by_14', 'ssn_count_7',
       'ssn_name_dob_count_0_by_14', 'ssn_name_count_7',
       'fulladdress_homephone_count_3', 'ssn_count_3', 'ssn_firstname_count_3',
       'ssn_dob_count_3', 'ssn_name_count_3', 'ssn_lastname_count_3',
       'ssn_name_dob_count_3', 'fraud_label'],
      dtype='object')

In [9]:
len(list(vars.columns))

31

In [14]:
vars.describe().T

,count,mean,std,min,25%,50%,75%,max
address_count_30,1000000.0,1.078171,0.664121,1.000000,1.0,1.0,1.0,30.0
fulladdress_count_30,1000000.0,1.064624,0.633831,1.000000,1.0,1.0,1.0,30.0
fulladdress_homephone_count_30,1000000.0,1.048207,0.507682,1.000000,1.0,1.0,1.0,30.0
ssn_dob_count_30,1000000.0,1.046097,0.496315,1.000000,1.0,1.0,1.0,34.0
name_dob_count_30,1000000.0,1.046295,0.496914,1.000000,1.0,1.0,1.0,34.0
ssn_count_30,1000000.0,1.050823,0.513331,1.000000,1.0,1.0,1.0,34.0
ssn_firstname_count_30,1000000.0,1.048903,0.499029,1.000000,1.0,1.0,1.0,34.0
ssn_name_count_30,1000000.0,1.048778,0.498892,1.000000,1.0,1.0,1.0,34.0
name_dob_count_14,1000000.0,1.032363,0.471763,1.000000,1.0,1.0,1.0,34.0
ssn_count_14,1000000.0,1.034591,0.477649,1.000000,1.0,1.0,1.0,34.0


In [10]:
Y_save = pd.DataFrame(vars.loc[:,'fraud_label'])
Y_save.head()

,fraud_label
0,0
1,1
2,0
3,0
4,0


## Scale and truncate field values

In [11]:
top_5_vars=['address_count_30', 'fulladdress_count_30',
       'fulladdress_homephone_count_30', 'ssn_dob_count_30',
       'name_dob_count_30']

In [12]:
top_10_vars=['address_count_30',
 'fulladdress_count_30',
 'fulladdress_homephone_count_30',
 'ssn_dob_count_30',
 'name_dob_count_30',
 'ssn_count_30',
 'ssn_firstname_count_30',
 'ssn_name_count_30',
 'name_dob_count_14',
 'ssn_count_14']

In [13]:
top_15_vars=['address_count_30',
 'fulladdress_count_30',
 'fulladdress_homephone_count_30',
 'ssn_dob_count_30',
 'name_dob_count_30',
 'ssn_count_30',
 'ssn_firstname_count_30',
 'ssn_name_count_30',
 'name_dob_count_14',
 'ssn_count_14',
 'ssn_name_dob_count_14',
 'ssn_lastname_count_14',
 'ssn_name_count_14',
 'ssn_dob_count_0_by_30',
 'name_dob_count_0_by_30']

In [14]:
top_20_vars=['address_count_30',
 'fulladdress_count_30',
 'fulladdress_homephone_count_30',
 'ssn_dob_count_30',
 'name_dob_count_30',
 'ssn_count_30',
 'ssn_firstname_count_30',
 'ssn_name_count_30',
 'name_dob_count_14',
 'ssn_count_14',
 'ssn_name_dob_count_14',
 'ssn_lastname_count_14',
 'ssn_name_count_14',
 'ssn_dob_count_0_by_30',
 'name_dob_count_0_by_30',
 'ssn_count_0_by_30',
 'ssn_name_dob_count_0_by_30',
 'homephone_count_3',
 'name_dob_count_0_by_14',
 'ssn_dob_count_0_by_14']

In [15]:
def x_scaling(vars_list):
    X_no_scaling = vars.filter(vars_list, axis=1)
    X=(X_no_scaling - X_no_scaling.mean()) / X_no_scaling.std()
    return X

In [16]:
X_top5=x_scaling(top_5_vars)

In [17]:
X_top5.describe().T

,count,mean,std,min,25%,50%,75%,max
address_count_30,1000000.0,-1.884392e-14,1.0,-0.117706,-0.117706,-0.117706,-0.117706,43.549034
fulladdress_count_30,1000000.0,-4.084356e-14,1.0,-0.101958,-0.101958,-0.101958,-0.101958,45.651540
fulladdress_homephone_count_30,1000000.0,-7.507017e-14,1.0,-0.094955,-0.094955,-0.094955,-0.094955,57.027380
ssn_dob_count_30,1000000.0,-4.867295e-14,1.0,-0.092879,-0.092879,-0.092879,-0.092879,66.397189
name_dob_count_30,1000000.0,-9.922335e-16,1.0,-0.093165,-0.093165,-0.093165,-0.093165,66.316650


In [17]:
X_top10=x_scaling(top_10_vars)

In [18]:
X_top15=x_scaling(top_15_vars)

In [19]:
X_top20=x_scaling(top_20_vars)

In [20]:
def clip_outlier(X):
    cols=X.columns
    X.loc[:,cols] = X[cols].clip(upper=Clip)
    X.loc[:,cols] = X[cols].clip(lower=-1*Clip)

In [21]:
clip_outlier(X_top5)

In [23]:
X_top5.shape

(1000000, 5)

In [62]:
X_top5.describe().T

,count,mean,std,min,25%,50%,75%,max
address_count_30,1000000.0,-0.016786,0.678129,-0.117706,-0.117706,-0.117706,-0.117706,10.0
fulladdress_count_30,1000000.0,-0.018176,0.631721,-0.101958,-0.101958,-0.101958,-0.101958,10.0
fulladdress_homephone_count_30,1000000.0,-0.018020,0.576299,-0.094955,-0.094955,-0.094955,-0.094955,10.0
ssn_dob_count_30,1000000.0,-0.017649,0.571302,-0.092879,-0.092879,-0.092879,-0.092879,10.0
name_dob_count_30,1000000.0,-0.017617,0.572884,-0.093165,-0.093165,-0.093165,-0.093165,10.0


In [22]:
clip_outlier(X_top10)

In [23]:
clip_outlier(X_top15)

In [24]:
clip_outlier(X_top20)

In [25]:
# separate data into modeling (traintest) and out of time
def trte_oot(X):
    X_trntst = X[0:833507]
    Y_trntst = Y_save[0:833507]
    X_oot = X[833507:]
    Y_oot = Y_save[833507:]
    return X_trntst,Y_trntst,X_oot,Y_oot

In [28]:
X_trntst_5,Y_trntst_5,X_oot_5,Y_oot_5=trte_oot(X_top5)

In [72]:
niter = 0
nitermax = 3
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig = X_oot_5.copy()
pd.options.mode.chained_assignment = None  # default='warn'

In [64]:
#X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_5, Y_trntst_5, test_size = .3)

In [70]:
#X_trntst_5.head()

,address_count_30,fulladdress_count_30,fulladdress_homephone_count_30,ssn_dob_count_30,name_dob_count_30
0,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165
1,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165
2,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165
3,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165
4,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165


In [76]:
print(X_oot_5.head(1))

        address_count_30  fulladdress_count_30  \
833507         -0.117706             -0.101958   

        fulladdress_homephone_count_30  ssn_dob_count_30  name_dob_count_30  
833507                       -0.094955         -0.092879          -0.093165  


In [ ]:
X_oot_5

In [74]:
%%time
# Logistic regression

for niter in range(nitermax):
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_5, Y_trntst_5, test_size = .3)

    model = LogisticRegression()
    
    X_oot = X_oot_orig_5.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()
    
    model.fit(X_trn, Y_trn.values.ravel())   


    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions 
    X_oot['fraud_label'] = Y_oot_5['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.46199524940617576 0.4767214942849178 0.45766974015088013
1 0.4627609108159393 0.47468531468531466 0.45766974015088013
2 0.4645791165614954 0.4706208425720621 0.45766974015088013
trn    0.463112
tst    0.474009
oot    0.457670
dtype: float64
Wall time: 5.02 s


In [82]:
%%time
# Logistic regression

for niter in range(nitermax):
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_5, Y_trntst_5, test_size = .3)

    model = LogisticRegression(C=0.1, penalty='l2',solver='lbfgs')
    
    X_oot = X_oot_orig_5.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()
    
    model.fit(X_trn, Y_trn.values.ravel())   


    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions 
    X_oot['fraud_label'] = Y_oot_5['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.4624175306314797 0.47570332480818417 0.45766974015088013
1 0.4679052042396094 0.4626038781163435 0.45766974015088013
2 0.46936114732724904 0.4591036414565826 0.45766974015088013
trn    0.466561
tst    0.465804
oot    0.457670
dtype: float64
Wall time: 5.13 s


In [29]:
X_trntst_10,Y_trntst_10,X_oot_10,Y_oot_10=trte_oot(X_top10)

In [30]:
niter = 0
nitermax = 3
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig_10 = X_oot_10.copy()
pd.options.mode.chained_assignment = None  # default='warn'

In [34]:
%%time
# Logistic regression

for niter in range(nitermax):
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_10, Y_trntst_10, test_size = .3)

    model = LogisticRegression(C=0.1,penalty='l1',solver='liblinear')
    
    X_oot = X_oot_orig_10.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()
    
    model.fit(X_trn, Y_trn.values.ravel())   


    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions 
    X_oot['fraud_label'] = Y_oot_5['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.47780050535434965 0.4837662337662338 0.46479463537300925
1 0.4837369420702754 0.471113591962043 0.46563285834031853
2 0.47904829545454547 0.4813149761168868 0.46437552388935455
trn    0.480195
tst    0.478732
oot    0.464934
dtype: float64
Wall time: 1min 48s


In [35]:
X_trntst_15,Y_trntst_15,X_oot_15,Y_oot_15=trte_oot(X_top15)

In [36]:
niter = 0
nitermax = 3
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig_15 = X_oot_15.copy()
pd.options.mode.chained_assignment = None  # default='warn'

In [39]:
%%time
# Logistic regression

for niter in range(nitermax):
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_15, Y_trntst_15, test_size = .3)

    model = LogisticRegression(C=0.1,penalty='l2',solver='liblinear')
    
    X_oot = X_oot_orig_15.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()
    
    model.fit(X_trn, Y_trn.values.ravel())   


    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions 
    X_oot['fraud_label'] = Y_oot_5['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.485833831440526 0.47528830313014825 0.4719195305951383
1 0.48056411471912774 0.488932474082376 0.47066219614417437
2 0.4850242689712324 0.47837078651685394 0.47150041911148366
trn    0.483807
tst    0.480864
oot    0.471361
dtype: float64
Wall time: 9.97 s


In [26]:
X_trntst_20,Y_trntst_20,X_oot_20,Y_oot_20=trte_oot(X_top20)

In [27]:
niter = 0
nitermax = 3
FDR3 = pd.DataFrame(np.zeros((nitermax,3)), columns=('trn', 'tst', 'oot'))
X_oot_orig_20 = X_oot_20.copy()
pd.options.mode.chained_assignment = None  # default='warn'

In [49]:
%%time
# Logistic regression

for niter in range(nitermax):
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_20, Y_trntst_20, test_size = .3)

    model = LogisticRegression(C=.1,penalty='l2',solver='liblinear')
    
    X_oot = X_oot_orig_20.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()
    
    model.fit(X_trn, Y_trn.values.ravel())   


    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions 
    X_oot['fraud_label'] = Y_oot_20['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5395933014354067 0.5346860433232794 0.5222129086336965
1 0.5428978284086864 0.5430167597765363 0.5213746856663872
2 0.5346958174904943 0.5499860763018658 0.5188600167644594
trn    0.539062
tst    0.542563
oot    0.520816
dtype: float64
Wall time: 13.7 s


In [75]:
X_trn

,address_count_30,fulladdress_count_30,fulladdress_homephone_count_30,ssn_dob_count_30,name_dob_count_30,predicted,fraud_label
178555,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165,0.006867,0
670564,1.388044,1.475749,1.874781,1.921972,1.919254,0.085739,0
310756,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165,0.006867,0
464136,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165,0.006867,0
182110,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165,0.006867,0
...,...,...,...,...,...,...,...
215504,1.388044,1.475749,-0.094955,1.921972,1.919254,0.105723,0
529002,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165,0.006867,0
55120,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165,0.006867,0
809187,-0.117706,-0.101958,-0.094955,-0.092879,-0.093165,0.006867,0


## Models. You can comment out any of these cells and just explore one model type. You can also just rerun that single cell multiple times as you explore different model hyperparameters.

In [56]:
%%time
# Single DT

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_20, Y_trntst_20, test_size = .3)

    model = DecisionTreeClassifier(max_depth=None,min_samples_split=2,min_samples_leaf=2,max_features='log2',criterion='gini')
 
    X_oot = X_oot_orig_20.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['fraud_label'] = Y_oot_20['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.55970591722993 0.5523223279238948 0.5347862531433362
1 0.5613111616222036 0.549890350877193 0.5356244761106455
2 0.5572873289708506 0.5605219322598556 0.5356244761106455
trn    0.559435
tst    0.554245
oot    0.535345
dtype: float64
Wall time: 4.14 s


In [61]:
%%time
# RF


for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_20, Y_trntst_20, test_size = .3)

    model = RandomForestClassifier(min_samples_leaf=2)
    X_oot = X_oot_orig_20.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['fraud_label'] = Y_oot_20['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5524126455906821 0.5711104926245477 0.5352053646269908
1 0.5598813056379822 0.5547180346175321 0.5360435875943
2 0.5605050023820867 0.5510938798116866 0.5347862531433362
trn    0.557600
tst    0.558974
oot    0.535345
dtype: float64
Wall time: 1min 45s


In [37]:
%%time
# BT

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_20, Y_trntst_20, test_size = .3)

    model = lgb.LGBMClassifier(n_estimators=50)

    X_oot = X_oot_orig_20.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['fraud_label'] = Y_oot_20['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5583871739650634 0.5579610852288298 0.5360435875943
1 0.5582142857142857 0.557527030773496 0.5356244761106455
2 0.5570414201183432 0.5591790834973293 0.5352053646269908
trn    0.557881
tst    0.558222
oot    0.535624
dtype: float64
Wall time: 8.69 s


In [42]:
%%time
# NN

for niter in range(nitermax):    
    X_trn, X_tst, Y_trn, Y_tst = train_test_split(X_trntst_20, Y_trntst_20, test_size = .3)

    model = MLPClassifier(solver='lbfgs',hidden_layer_sizes=(50,100,50))

    X_oot = X_oot_orig_20.copy()
    X_trn_save = X_trn.copy()
    Y_trn_save = Y_trn.copy()

    model.fit(X_trn, Y_trn.values.ravel())   

    predictions = model.predict_proba(X_trn_save)[:,1]
    X_trn['predicted'] = predictions
    X_trn['fraud_label'] = Y_trn_save['fraud_label']
    topRows = int(round(X_trn.shape[0]*0.03))
    temp = X_trn.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'trn'] = sum(needed)/sum(X_trn.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_tst)[:,1]
    X_tst['predicted']=predictions
    X_tst['fraud_label'] = Y_tst['fraud_label']
    topRows = int(round(X_tst.shape[0]*0.03))
    temp = X_tst.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'tst'] = sum(needed)/sum(X_tst.loc[:,'fraud_label'])

    predictions = model.predict_proba(X_oot)[:,1]
    X_oot['predicted']=predictions
    X_oot['fraud_label'] = Y_oot_20['fraud_label']
    topRows = int(round(X_oot.shape[0]*0.03))
    temp = X_oot.sort_values('predicted',ascending=False).head(topRows)
    needed = temp.loc[:,'fraud_label']
    FDR3.loc[niter, 'oot'] = sum(needed)/sum(X_oot.loc[:,'fraud_label'])
    print(niter, FDR3.loc[niter, 'trn'],FDR3.loc[niter, 'tst'],FDR3.loc[niter, 'oot'])
    
print(FDR3.mean())

0 0.5547531668047828 0.5606741573033708 0.5347862531433362
1 0.5575959933222037 0.5537144435238884 0.5347862531433362
2 0.5574023507063991 0.5583147321428571 0.5347862531433362
trn    0.556584
tst    0.557568
oot    0.534786
dtype: float64
Wall time: 39min 5s
Compiler : 135 ms


In [ ]:
print('duration: ', datetime.now() - start_time)

duration:  0:03:48.900114
